# Here is the breakdown of all of the various checkers I made
## Cert checker 
It does need work to figure out a better scoring metric, but this is something that ChatGPT suggested prob does need some updates

In [8]:
# Here are the main libraries you will be needing
import ssl # performing TLS handshake, and getting the SSL cert 
import socket # low-level network connections 
from datetime import datetime, timezone #checking dates for certs if needed

In [9]:
#building a list of people we can trust by default  https://developer.visa.com/pages/trusted_certifying_authorities
TRUSTED_CAS = {
    "DigiCert","GlobalSign", "GeoTrust","Entrust","GoDaddy","Comodo", "USERTrust", "Sectigo",
    "Verisign",  "SecureTrust",  "Certum",  "QuoVadis",  "AddTrust",  "Actalis"
}

# the logic is to take a domain, connect to it to check cert, and score it 
def get_ssl_info(domain): 
    port = 443
    context = ssl.create_default_context()

    try: # help to avoid crashing - making it possible to loop this through multiple domains 
        with socket.create_connection((domain, port), timeout=5) as sock:
            with context.wrap_socket(sock, server_hostname=domain) as ssock:
                cert = ssock.getpeercert()

                exp_date = datetime.strptime(
                    cert['notAfter'],
                    '%b %d %H:%M:%S %Y %Z'
                ).replace(tzinfo=timezone.utc)

                start_date = datetime.strptime(
                    cert['notBefore'],
                    '%b %d %H:%M:%S %Y %Z'
                ).replace(tzinfo=timezone.utc)

                days_remaining = (exp_date - datetime.now(timezone.utc)).days

                issuer = dict(x[0] for x in cert['issuer'])
                subject = dict(x[0] for x in cert['subject'])

                issuer_org = issuer.get('organizationName', '')
                issuer_trusted = any(ca in issuer_org for ca in TRUSTED_CAS)
                
                return {
                    "valid_from": start_date,
                    "valid_to": exp_date,
                    "issuer": issuer_org,
                    "subject": subject.get('commonName'),
                    "days_remaining": days_remaining,
                    "is_self_signed": issuer == subject,
                    "is_expired": days_remaining < 0,
                    "issuer_trusted": issuer_trusted,
                    "certificate_score": max(score, 0)
                }

    except Exception as e: # allows for help with debuging 
        return {"error": str(e)}

In [10]:
def calculate_trust_score(ssl_info):
    if "error" in ssl_info:
        return 0, ["SSL connection failed"]

    score = 50  # start neutral instead of 0
    flags = []

    # Expiration logic
    if ssl_info["days_remaining"] > 90:
        score += 20
    elif ssl_info["days_remaining"] > 30:
        score += 10
    elif ssl_info["days_remaining"] > 0:
        flags.append("Certificate expiring soon")
    else:
        score -= 50
        flags.append("Certificate expired")

    # Self-signed check
    if ssl_info["is_self_signed"]:
        score -= 40
        flags.append("Self-signed certificate")

    # Trusted CA check
    if ssl_info["issuer_trusted"]:
        score += 20
    else:
        score -= 20
        flags.append("Untrusted certificate authority")

    # Final clamp
    score = max(0, min(score, 100))

    return score, flags

In [12]:
domain = input("Insert the website: ")

ssl_info = get_ssl_info(domain)

trust_score, flags = calculate_trust_score(ssl_info)

print("Trust Score:", trust_score)
print("Flags:", flags)

Insert the website:  www.amazon.com


Trust Score: 0
Flags: ['SSL connection failed']


## Some important notes 
This code doesnt work fully if you use it on a cloud environment. Best way to get it to work will be locally on your computer or your own server. But this should be a solid basis for adding into the code

Another note would be to make the websites be either lowercase or uppercase, but I don't think it matters as much given the code searches up the domains itself.

In [14]:
# another version that accounts for http redirects
def check_http_redirect(domain):
    try:
        with socket.create_connection((domain, 80), timeout=5) as sock:
            request = f"GET / HTTP/1.1\r\nHost: {domain}\r\nConnection: close\r\n\r\n"
            sock.sendall(request.encode())

            response = sock.recv(4096).decode(errors="ignore")

            # Check for HTTPS redirect
            if "301" in response or "302" in response:
                if "Location: https://" in response:
                    return True

            return False

    except:
        return False

# modified ssl function 
import ssl
import socket
from datetime import datetime, timezone

TRUSTED_CAS = {
    "DigiCert", "GlobalSign", "GeoTrust", "Entrust",
    "GoDaddy", "Comodo", "USERTrust", "Sectigo",
    "Verisign", "SecureTrust", "Certum",
    "QuoVadis", "AddTrust", "Actalis"
}

def is_port_open(domain, port):
    try:
        with socket.create_connection((domain, port), timeout=3):
            return True
    except:
        return False


def get_ssl_info(domain):
    port_443_open = is_port_open(domain, 443)
    port_80_open = is_port_open(domain, 80)
    http_redirects = check_http_redirect(domain) if port_80_open else False

    if not port_443_open:
        return {
            "error": "HTTPS not supported",
            "port_80_open": port_80_open,
            "http_redirects_to_https": http_redirects,
            "risk_flag": "No SSL detected"
        }

    context = ssl.create_default_context()

    try:
        with socket.create_connection((domain, 443), timeout=5) as sock:
            with context.wrap_socket(sock, server_hostname=domain) as ssock:
                cert = ssock.getpeercert()

                exp_date = datetime.strptime(
                    cert['notAfter'],
                    '%b %d %H:%M:%S %Y %Z'
                ).replace(tzinfo=timezone.utc)

                days_remaining = (exp_date - datetime.now(timezone.utc)).days

                issuer = dict(x[0] for x in cert['issuer'])
                subject = dict(x[0] for x in cert['subject'])

                issuer_org = issuer.get('organizationName', '')
                issuer_trusted = any(ca in issuer_org for ca in TRUSTED_CAS)

                score = 100

                if days_remaining < 0:
                    score -= 50
                elif days_remaining < 30:
                    score -= 20

                if issuer == subject:
                    score -= 40

                if not issuer_trusted:
                    score -= 30

                # HTTP logic scoring
                if port_80_open and not http_redirects:
                    score -= 15  # bigger penalty
                elif port_80_open and http_redirects:
                    score -= 3   # small penalty

                return {
                    "issuer": issuer_org,
                    "days_remaining": days_remaining,
                    "issuer_trusted": issuer_trusted,
                    "port_80_open": port_80_open,
                    "http_redirects_to_https": http_redirects,
                    "certificate_score": max(score, 0)
                }

    except Exception as e:
        return {"error": str(e)}

Insert the website:  WWW.google.com


KeyError: 'is_self_signed'